<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Piez%C3%B3metros_Casagrande_Versi%C3%B3n_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MONITOREO INSTRUMENTACIÓN PRESAS

#**Carga Masiva de datos Crudos**

In [2]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

In [1]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


## 📊 Visualización Interactiva – Piezómetros Casagrande

Este panel permite:
- Filtrar por **margen**, piezómetro, variable y año**
- Seleccionar el **estilo gráfico** (curvas, líneas, puntos, área)
- Definir el **tamaño**, **grosor** y la **paleta de colores**
- Exportar la gráfica en múltiples formatos (`PNG`, `JPG`, `PDF`, `SVG`, `HTML`)
- Descargar automáticamente la gráfica a tu PC desde Colab

🔧 Para exportar imágenes, asegurate de ejecutar:
```bash
!pip install -U plotly==6.1.1 kaleido==0.2.1


In [ ]:
import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecutá: pip install plotly ipywidgets")

# === WIDGETS Y CONTROLES ===
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")

estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:")

tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:")

grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:")

paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly, "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis, "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno, "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold, "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:")

output = widgets.Output()
output_guardar = widgets.Output()

boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")

ruta_text = widgets.Text(value="grafica_casagrande", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:")

controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# === FUNCIONES ===

def obtener_df(origen):
    try:
        return datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
    except:
        return pd.DataFrame()

def actualizar_margen(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'MARGEN' not in df.columns:
        margen_dropdown.options = []
        return
    margenes = sorted(df['MARGEN'].dropna().unique())
    margen_dropdown.options = margenes
    if margenes:
        margen_dropdown.value = margenes[0]

def actualizar_piezometros(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    margen = margen_dropdown.value
    piezos = sorted(df[df['MARGEN'] == margen]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + list(piezos)
    piezometro_dropdown.value = "Todos"

def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluir = ['FECHA', 'MARGEN', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        margen = margen_dropdown.value
        piezo = piezometro_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        df_plot = df[df['MARGEN'] == margen]
        if piezo != "Todos":
            df_plot = df_plot[df_plot['PIEZOMETRO'] == piezo]
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
        df_plot = df_plot.dropna(subset=['FECHA', variable, 'PIEZOMETRO'])

        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['PIEZOMETRO'].unique())
        color_map = {pz: paleta[i % len(paleta)] for i, pz in enumerate(instrumentos)}

        for pz in instrumentos:
            datos_pz = df_plot[df_plot['PIEZOMETRO'] == pz]
            line_args = dict(width=grosor, color=color_map[pz])
            marker_args = dict(color=color_map[pz])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None
            stackgroup = "one" if estilo == "Área apilada" else None

            fig.add_trace(go.Scatter(
                x=datos_pz['FECHA'],
                y=datos_pz[variable],
                mode=modo_graf,
                name=pz,
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill,
                stackgroup=stackgroup
            ))

        fig.update_layout(
            width=ancho, height=alto,
            title=f"{margen} – {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Piezómetro",
            hovermode="x unified"
        )
        fig.show()

def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instalá kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
origen_dropdown.observe(actualizar_margen, names='value')
margen_dropdown.observe(actualizar_piezometros, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')

boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === INICIALIZAR ===
actualizar_margen()
actualizar_variables_y_anios()
actualizar_piezometros()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Piezómetros Casagrande</h2>"))
display(origen_dropdown)
display(widgets.HBox([margen_dropdown, piezometro_dropdown]))
display(widgets.HBox([variable_dropdown, anio_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)


Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()

In [ ]:
# === VISUALIZACIÓN PIEZÓMETROS CASAGRANDE INTERACTIVA CON DESCARGA ===

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# --- Widgets y salidas ---
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly, "D3": pc.qualitative.D3, "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis, "Inferno": pc.sequential.Inferno, "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold, "Set1": pc.qualitative.Set1, "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_casagrande", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# --- Función para obtener DataFrame según origen ---
def obtener_df(origen):
    try:
        return datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
    except:
        return pd.DataFrame()

# --- Actualizar MARGENES disponibles ---
def actualizar_margenes(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'MARGEN' not in df.columns:
        margen_dropdown.options = []
        return
    margenes = sorted(df['MARGEN'].dropna().unique())
    margen_dropdown.options = margenes
    if margenes:
        margen_dropdown.value = margenes[0]

# --- Actualizar PIEZÓMETROS disponibles según MARGEN ---
def actualizar_piezometros(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    margen = margen_dropdown.value
    piezos = sorted(df[df['MARGEN'] == margen]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + list(piezos)
    piezometro_dropdown.value = "Todos"

# --- Actualizar VARIABLES y AÑOS ---
def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluir = ['FECHA', 'MARGEN', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        margen = margen_dropdown.value
        piezo = piezometro_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        df_plot = df[df['MARGEN'] == margen]
        if piezo != "Todos":
            df_plot = df_plot[df_plot['PIEZOMETRO'] == piezo]
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
        df_plot = df_plot.dropna(subset=['FECHA', variable, 'PIEZOMETRO'])

        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['PIEZOMETRO'].unique())
        color_map = {pz: paleta[i % len(paleta)] for i, pz in enumerate(instrumentos)}

        for pz in instrumentos:
            datos_pz = df_plot[df_plot['PIEZOMETRO'] == pz]
            line_args = dict(width=grosor, color=color_map[pz])
            marker_args = dict(color=color_map[pz])
            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None
            stackgroup = "one" if estilo == "Área apilada" else None

            fig.add_trace(go.Scatter(
                x=datos_pz['FECHA'],
                y=datos_pz[variable],
                mode=modo_graf,
                name=pz,
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill,
                stackgroup=stackgroup
            ))

        fig.update_layout(
            width=ancho, height=alto,
            title=f"{margen} – {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Piezómetro",
            hovermode="x unified"
        )
        fig.show()

# --- Guardar y descargar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Falta Kaleido. Ejecutá:\n!pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")

            # Descargar automáticamente
            from google.colab import files
            files.download(nombre)
        except Exception as e:
            print("❌ Error al guardar:", e)

# --- Eventos ---
origen_dropdown.observe(actualizar_margenes, names='value')
margen_dropdown.observe(actualizar_piezometros, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Inicializar ---
actualizar_margenes()
actualizar_variables_y_anios()
actualizar_piezometros()

# --- Mostrar ---
display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Piezómetros Casagrande</h2>"))
display(origen_dropdown)
display(widgets.HBox([margen_dropdown, piezometro_dropdown]))
display(widgets.HBox([variable_dropdown, anio_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)


Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()

#Script Actualizado que agrega la funcionalidad de Nivel Freatico con respecto al anterior, que funciona OK pero no tiene NK

In [4]:
# === VISUALIZACIÓN PIEZÓMETROS ELÉCTRICOS CON SELECTOR DE ORIGEN, PROGRESIVA Y PIEZÓMETRO ===

import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# --- Widgets y salidas ---
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)",
    description="Estilo:"
)

tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500),
    description="Tamaño:"
)

grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2,
    description="Grosor:"
)

paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly,
    description="Paleta:"
)

output = widgets.Output()
output_guardar = widgets.Output()
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica_piezometros", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png",
    description="Formato:"
)

controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

# --- Función para obtener DataFrame según origen ---
def obtener_df(origen):
    try:
        return datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
    except:
        return pd.DataFrame()

# --- Actualizar PROGRESIVAS disponibles ---
def actualizar_progresivas(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'PROGRESIVA' not in df.columns:
        progresiva_dropdown.options = []
        return
    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown.options = progresivas
    if progresivas:
        progresiva_dropdown.value = progresivas[0]

# --- Actualizar PIEZÓMETROS disponibles según PROGRESIVA ---
def actualizar_piezometros(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    prog = progresiva_dropdown.value
    piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + list(piezos)
    piezometro_dropdown.value = "Todos"

# --- Actualizar VARIABLES y AÑOS ---
def actualizar_variables_y_anios(change=None):
    df = obtener_df(origen_dropdown.value)
    if df.empty:
        variable_dropdown.options = []
        anio_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluir = ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']
    variables = [c for c in df.columns if c not in columnas_excluir]
    variable_dropdown.options = variables
    if variables:
        variable_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

# --- Graficar ---
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = obtener_df(origen_dropdown.value).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        prog = progresiva_dropdown.value
        piezo = piezometro_dropdown.value
        variable = variable_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = tamanio_dropdown.value
        grosor = grosor_dropdown.value
        paleta = paleta_dropdown.value

        df_plot = df[df['PROGRESIVA'] == prog].copy()
        if piezo != "Todos":
            df_plot = df_plot[df_plot['PIEZOMETRO'] == piezo]
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]

        # --- Conversión especial para COTA_NF ---
        if variable == "COTA_NF":
            df_plot[variable] = pd.to_numeric(df_plot[variable], errors="coerce")

        df_plot = df_plot.dropna(subset=['FECHA', variable, 'PIEZOMETRO'])

        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['PIEZOMETRO'].unique())
        color_map = {pz: paleta[i % len(paleta)] for i, pz in enumerate(instrumentos)}

        for pz in instrumentos:
            datos_pz = df_plot[df_plot['PIEZOMETRO'] == pz]
            line_args = dict(width=grosor, color=color_map[pz])
            marker_args = dict(color=color_map[pz])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }
            modo_graf, line_shape = modo[estilo]
            fill = "tozeroy" if "Área" in estilo else None
            stackgroup = "one" if estilo == "Área apilada" else None

            fig.add_trace(go.Scatter(
                x=datos_pz['FECHA'],
                y=datos_pz[variable],
                mode=modo_graf,
                name=pz,
                line_shape=line_shape,
                line=line_args,
                marker=marker_args,
                fill=fill,
                stackgroup=stackgroup
            ))

        fig.update_layout(
            width=ancho, height=alto,
            title=f"{prog} – {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Piezómetro",
            hovermode="x unified"
        )
        fig.show()

# --- Guardar gráfica ---
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        ext = formato_dropdown.value
        nombre = ruta_text.value
        if not nombre.lower().endswith(ext):
            nombre += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Generá una gráfica primero.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(nombre, format=ext[1:])
            elif ext == ".html":
                fig.write_html(nombre)
            print(f"✅ Guardado: {os.path.abspath(nombre)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# --- Eventos ---
origen_dropdown.observe(actualizar_progresivas, names='value')
progresiva_dropdown.observe(actualizar_piezometros, names='value')
origen_dropdown.observe(actualizar_variables_y_anios, names='value')

boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- Inicializar y mostrar controles ---
actualizar_progresivas()
actualizar_variables_y_anios()
actualizar_piezometros()

display(HTML("<h2 style='color:#1866a3;'>Visualización Interactiva – Piezómetros Eléctricos</h2>"))
display(origen_dropdown)
display(widgets.HBox([progresiva_dropdown, piezometro_dropdown]))
display(widgets.HBox([variable_dropdown, anio_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown]))
display(widgets.HBox([grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(controles_guardar)
display(boton_guardar)
display(output_guardar)


Dropdown(description='Origen:', options=('CSV', 'XLSX'), value='CSV')

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Button(button_style='info', description='Guardar gráfica', style=ButtonStyle())

Output()